In [1]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import shutil
import numpy as np
import random

from ultralytics import YOLO
from PIL import Image

# EDA

In [2]:
# Metadata para ordenar el entrenamiento
metadata_path = 'datasets\Labeled Stomatal Images.csv'
metadata = pd.read_csv(metadata_path)

In [3]:
metadata.head()

,FileName,Species,ScientificName,Magnification,Witdh,Heigth,Resolution
0,STMHD0001,Nuttall oak,Quercus texana Buckley,100,1024,768,118
1,STMHD0002,Nuttall oak,Quercus texana Buckley,100,1024,768,118
2,STMHD0003,Nuttall oak,Quercus texana Buckley,100,1024,768,118
3,STMHD0004,Nuttall oak,Quercus texana Buckley,100,1024,768,118
4,STMHD0005,Nuttall oak,Quercus texana Buckley,100,1024,768,118


In [4]:
metadata.Species.unique()

array(['Nuttall oak', 'Shumard oak', 'American elm', 'American Holly', 'Cherrybark oak', 'Shagbark hickory', 'Swamp chestnut oak', 'Water oak', 'Willow oak', 'Winged elm', 'Ash', 'Deerberry', 'Leatherwood', 'Red maple', 'Post oak', 'Black gum', 'Willow', 'Poplar'], dtype=object)

In [5]:
# Calcular el número total de imágenes
total_images = len(metadata)

# Calcular la media y desviación estándar de Width y Height
mean_width = metadata['Witdh'].mean()
std_width = metadata['Witdh'].std()
mean_height = metadata['Heigth'].mean()
std_height = metadata['Heigth'].std()

# Número de especies únicas
unique_species = metadata['Species'].nunique()

# Frecuencia de cada especie
species_frequency = metadata['Species'].value_counts()

# Frecuencia de resoluciones (Width x Height)
resolution_frequency = metadata.groupby(['Witdh', 'Heigth']).size()

# Media de la magnificación
mean_magnification = metadata['Magnification'].mean()

# Frecuencia de magnificación
magnification_frequency = metadata['Magnification'].value_counts()

# Print results
print("Total de imágenes:", total_images)
print("Witdh promedio:", mean_width, "Desviación estándar del ancho:", std_width)
print("Heigth promedio:", mean_height, "Desviación estándar de la altura:", std_height)
print("Número de especies únicas:", unique_species)
print("Frecuencia de cada especie:", species_frequency)
print("Frecuencia de resoluciones:", resolution_frequency)
print("Magnificación promedio:", mean_magnification)
print("Frecuencia de magnificaciones:", magnification_frequency)


Total de imágenes: 10715
Witdh promedio: 1407.319085394307 Desviación estándar del ancho: 495.5886844096622
Heigth promedio: 1055.4893140457302 Desviación estándar de la altura: 371.6915133072467
Número de especies únicas: 18
Frecuencia de cada especie: Species
Poplar                3068
Nuttall oak           2975
Shumard oak           1612
Water oak              653
Willow oak             568
Cherrybark oak         286
Winged elm             205
Ash                    200
Post oak               196
American elm           185
Leatherwood            130
Swamp chestnut oak     129
American Holly         100
Red maple              100
Shagbark hickory        91
Deerberry               85
Willow                  82
Black gum               50
Name: count, dtype: int64
Frecuencia de resoluciones: Witdh  Heigth
1024   768       6704
2048   1536      4011
dtype: int64
Magnificación promedio: 270.32197853476436
Frecuencia de magnificaciones: Magnification
200    5740
400    4170
100     805
Nam

In [6]:
path = 'datasets/train/labels'

line_counts = []

for filename in os.listdir(path):
    if filename.endswith('.txt'): 
        with open(os.path.join(path, filename), 'r') as file:
            lines = file.readlines()
            line_counts.append(len(lines)) 

if line_counts:
    mean_boxes = np.mean(line_counts)
    variance = np.var(line_counts)
    std_dev = variance ** 0.5
    print(f'Total de bounding boxes: {sum(line_counts)}')
    print(f'Media de bounding boxes por imagen: {mean_boxes}')
    print(f'Desviación estándar de bounding boxes por imagen: {std_dev}')
else:
    print("No se encontraron archivos .txt en el directorio.")

Total de bounding boxes: 683371
Media de bounding boxes por imagen: 79.72130191320579
Desviación estándar de bounding boxes por imagen: 68.90806835249873


# Preprocesamiento

In [ ]:
#Mediciones 1 plant-out
def onePlantOut(plant_name):
  main_folder = 'datasets'

  train_images_folder = os.path.join(main_folder, 'train/images')
  train_labels_folder = os.path.join(main_folder, 'train/labels')
  val_images_folder = os.path.join(main_folder, 'val/images')
  val_labels_folder = os.path.join(main_folder, 'val/labels')

  folders = [train_images_folder, train_labels_folder, val_images_folder, val_labels_folder]

  # Sino existen los directorios
  for folder in folders:
      if not os.path.exists(folder):
          os.makedirs(folder)

  # Seleccionar la planta a medir su rendimiento
  plant = plant_name
  trainFiles = metadata[metadata['Species'] != plant]['FileName'].tolist()
  valFiles = metadata[metadata['Species'] == plant]['FileName'].tolist()

  for folder in folders:
    for file in os.listdir(folder):
      if file.split(".")[0] in trainFiles:
        if file.endswith(".jpg"):
          shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'train/images', file))
        elif file.endswith(".txt"):
          shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'train/labels', file))

  for folder in folders:
    for file in os.listdir(folder):
      if file.split(".")[0] in valFiles:
        if file.endswith(".jpg"):
          shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'val/images', file))
        elif file.endswith(".txt"):
          shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'val/labels', file))

def holdOut(random_seed = 42, test_size = 0.2):
    random.seed(random_seed)
    main_folder = 'datasets'

    train_images_folder = os.path.join(main_folder, 'train/images')
    train_labels_folder = os.path.join(main_folder, 'train/labels')
    val_images_folder = os.path.join(main_folder, 'val/images')
    val_labels_folder = os.path.join(main_folder, 'val/labels')

    folders = [train_images_folder, train_labels_folder, val_images_folder, val_labels_folder]

  # Sino existen los directorios
    for folder in folders:
        if not os.path.exists(folder):
            os.makedirs(folder)

    # Obtener todos los archivos de la columna 'FileName'
    all_files = metadata['FileName'].tolist()

    # Mezclar aleatoriamente los archivos
    random.shuffle(all_files,)

    split_idx = int((1-test_size) * len(all_files))
    trainFiles = all_files[:split_idx]
    valFiles = all_files[split_idx:]

    for folder in folders:
        for file in os.listdir(folder):
            if file.split(".")[0] in trainFiles:
                if file.endswith(".jpg"):
                    shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'train/images', file))
                elif file.endswith(".txt"):
                    shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'train/labels', file))
    
    for folder in folders:
        for file in os.listdir(folder):
            if file.split(".")[0] in valFiles:
                if file.endswith(".jpg"):
                    shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'val/images', file))
                elif file.endswith(".txt"):
                    shutil.move(os.path.join(folder, file), os.path.join(main_folder, 'val/labels', file))



In [ ]:
holdOut(random_seed = 42, test_size = 0.2)
# onePlantOut("Nuttall oak")

In [ ]:
PATH_images = "datasets/train/images"
PATH_labels = "datasets/train/labels"
plant_type = '/STMHD'
number = "0005"
image_path = f"{PATH_images}{plant_type}{number}.jpg"
txt_path = f"{PATH_labels}{plant_type}{number}.txt"

def show_image(image_path, txt_path):
    img = cv2.imread(image_path)
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # Leer y mostrar el contenido del archivo .txt
    with open(txt_path, 'r') as file:
        annotations = file.readlines()

    for annotation in annotations:
        class_id, x_center, y_center, width, height = map(float, annotation.split())
        x = (x_center - width / 2) * img.shape[1]
        y = (y_center - height / 2) * img.shape[0]
        w = width * img.shape[1]
        h = height * img.shape[0]
        rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()
show_image(image_path, txt_path)

# YOLO

In [ ]:
modelSimple = YOLO('Modelos\yolov10\yolov10n.pt')

# Entrenar el modelo
modelSimple.train(data='YOLO8INFO.yaml', epochs=30, imgsz=640, batch=16, device=0) #Simple
modelSimple.save('Modelos\yolov10\HoldOutSimpleYolov10n.pt')

In [14]:
modelAdjust = YOLO('Modelos\yolov10\HoldOutSimpleYolov10n.pt')
# Entrenar el modelo con hiperparámetros ajustados
modelAdjust.train(
    data='YOLO8INFO.yaml',     # Archivo de configuración de los datos
    epochs=100,                 # Aumentar el número de épocas para una mayor convergencia
    patience = 20,             # Number of epochs to wait without improvement in validation metrics 
    imgsz=640,                 # Tamaño de las imágenes para mejorar la precisión
    batch=16,                  # Tamaño del lote mayor para un entrenamiento más rápido (ajustar según la GPU)
    device=0,                  # Especificar el dispositivo GPU
    lr0=0.01,                  # Tasa de aprendizaje inicial más alta
    lrf=0.2,                   # Factor de reducción de la tasa de aprendizaje
    momentum=0.937,            # Momento optimizado para estabilizar el entrenamiento
    weight_decay=0.0005,       # Decaimiento del peso para evitar el sobreajuste
    warmup_epochs=5.0,         # Épocas de calentamiento para estabilizar el modelo al inicio
    warmup_momentum=0.8,       # Momento de calentamiento
    warmup_bias_lr=0.1,        # Tasa de aprendizaje inicial para el sesgo
    box= 7,                    # Ponderación de la pérdida de la caja
    cls=0.5,                   # Ponderación de la pérdida de clasificación
    dfl=1.0,                   # Ponderación de la pérdida de distribución de la etiqueta
    hsv_h=0.015,               # Augmentación de color - ajuste de tono
    hsv_s=0.7,                 # Augmentación de color - ajuste de saturación
    hsv_v=0.4,                 # Augmentación de color - ajuste de valor
    degrees= 60,               # Augmentación - rotación de la imagen
    translate=0.1,             # Augmentación - traducción de la imagen
    scale=0.5,                 # Augmentación - escalado de la imagen
    shear=0.0,                 # Augmentación - cizallamiento
    flipud=0.0,                # Volteo vertical (no comúnmente usado)
    fliplr=0.5,                # Volteo horizontal (común para imágenes de objetos)
    mosaic=0.2,                # Habilitar la mezcla de mosaico (aumenta el conjunto de datos)
    mixup=0.4,                 # Habilitar mixup para mezclar las imágenes
    label_smoothing=0.1,       # Suavización de etiquetas para evitar sobreajuste
    dropout = 0.1,             # Tasa de droput para regularizar
)
modelAdjust.save('Modelos\yolov10\HoldOutSimpleYolov10nRETRAIN.pt')

New https://pypi.org/project/ultralytics/8.3.31 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.100  Python-3.10.9 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: task=detect, mode=train, model=Modelos\yolov10\HoldOutSimpleYolov10n.pt, data=YOLO8INFO.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, em

train: Scanning C:\Escritorio\Semestre 10\Stomatal-Density-with-Deep-Learning\datasets\train\labels... 8572 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8572/8572 [00:10<00:00, 844.44it/s] 


train: New cache created: C:\Escritorio\Semestre 10\Stomatal-Density-with-Deep-Learning\datasets\train\labels.cache


val: Scanning C:\Escritorio\Semestre 10\Stomatal-Density-with-Deep-Learning\datasets\val\labels.cache... 2143 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2143/2143 [00:00<?, ?it/s]


Plotting labels to runs\detect\train9\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 95 weight(decay=0.0), 108 weight(decay=0.0005), 107 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train9
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

    100/100      6.32G      1.242     0.5785      1.106       1011        640: 100%|██████████| 536/536 [02:31<00:00,  3.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:23<00:00,  2.84it/s]

                   all       2143     163185      0.963      0.959       0.99      0.808



100 epochs completed in 5.875 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 5.7MB
Optimizer stripped from runs\detect\train9\weights\best.pt, 5.7MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics YOLOv8.2.100  Python-3.10.9 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
YOLOv10n summary (fused): 285 layers, 2,695,196 parameters, 0 gradients, 8.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [01:01<00:00,  1.10it/s]


                   all       2143     163185      0.964      0.959       0.99      0.808
                stoma1       2143      99280      0.972      0.958      0.991      0.828
                stoma2       1807      63905      0.955       0.96      0.989      0.787
Speed: 0.1ms preprocess, 2.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\train9


In [ ]:
def plot_detections(results):
    img_rgb = cv2.cvtColor(results.orig_img, cv2.COLOR_BGR2RGB)

    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img_rgb)
    for box, score in zip(results.boxes.xyxy, results.boxes.conf):
        x1, y1, x2, y2 = box
        # Coordenadas deben ser ints
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        width, height = x2 - x1, y2 - y1
        rect = patches.Rectangle((x1, y1), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

        # Mostrar el score como etiqueta en la esquina superior izquierda de la caja
        label = f"{score:.2f}"  # Formatear el score a 2 decimales
        ax.text(x1, y1, label, color='white', fontsize=5, verticalalignment='top', bbox={'color': 'r', 'pad': 0})
        
    plt.show()

In [ ]:
image_path = "datasets/val/images/STMHD0007.jpg"

results = modelAdjust.predict(image_path)[0]

plot_detections(results)

## Entrenamiento 1-plantOut

In [ ]:
species = metadata.Species.unique()
for s in species:
    modelSimple = YOLO('Modelos\yolov10\yolov10n.pt')
    # Ordenamos por cada planta
    onePlantOut(s)
    modelSimple.train(data='YOLO8INFO.yaml', epochs=30, imgsz=640, batch= 16, device=0) 
    modelSimple.save(f'Modelos\yolov10\{s}SimpleYolov10n.pt')